# Capstone Project - The Battle of Neighborhoods (Week 1)

# Introduction/Business Problem

Rajasthan state of India is famous for its historical places and culture. Many cities across the state have witnessed great history and monuments in those cities are places of great attraction for tourists. So in this project we have decided to compare two cities of Rajasthan : Jaipur and Ajmer. And the target of the study is to compare how similar are the two cities and what kind of business is favourable in each of the two cities.

# Data

We will be using Foursquare API for retriving geographical data for exploring the neighbourhoods of the two cities. We will be using the Lat long of the two cities to query the API and fetch the results and analyse it for comparing the two cities.

In [ ]:
#!pip install folium

In [3]:
from geopy.geocoders import Nominatim 
import folium

In [4]:
address = 'Jaipur, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [5]:
print('the coordinates of jaipur are lat: {} and long: {}'.format(latitude, longitude))

the coordinates of jaipur are lat: 26.916194 and long: 75.820349


In [40]:
CLIENT_ID = '44TLKI2RQKYUZGS0PUP2EG3YD1U2FQU1DJPMG4W4SEVSXR3V' 
CLIENT_SECRET = 'EEPPCM3JRVRDSUFJ0QW2C3UYRQIIA15JB5OKTHFBZH4LKBEK'
VERSION = '20200104'
LIMIT = 100
radius = 500

# type your answer here
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,
                                                                                                                                radius, LIMIT)

In [41]:
import requests
results = requests.get(url).json()

In [42]:
import pandas as pd

neighborhoods_data = results['response']['venues']# define the dataframe columns
column_names = ['Name', 'Categories', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [43]:
for data in neighborhoods_data:
    
    neighborhood_name = data['name']
    neighborhood_lat = data['location']['lat']
    neighborhood_lon = data['location']['lng']
    if(len(data['categories'])):
        neighborhood_categories = data['categories'][0]['name']
    else:
        neighborhood_categories = None
    neighborhoods = neighborhoods.append({'Name': neighborhood_name,
                                          'Categories': neighborhood_categories,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [45]:
# create map of Manhattan using latitude and longitude values
map_jaipur = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jaipur)  
    
map_jaipur

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
jaipur_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )